# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [7]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
url = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг']
params = {
    'search_query': 'python'
}

In [51]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<meta content="444736788986613" property="fb:app_id">
<meta content="website" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="Хабр" property="og:site_name"/>
<link href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" rel="image_src"/>
<meta content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="628" property="og:image:height"/>
<meta content="Все публикации подряд / Хабр" property="og:title"/>
<meta content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом." property="og:description"/>
<meta content="71593b225aeafc4e" name="yandex-verification"/>
<meta content="uns

In [9]:
#news_blocks = soup.find_all('a', class_='post__title_link')
news_blocks = soup.find_all('h2', class_='post__title')
news_blocks


[<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/565904/">Почему иногда React/Redux в текущем состоянии give me creeps</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/565900/">Хроника будущего: как будет эволюционировать человек следующий миллиард лет?</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/565892/">Covid fake FAQ___draft_final_4 (окончательное доказательство)</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/564992/">Королевские рефакторинги с Roslyn</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/englishdom/blog/565786/">5 способов перевода английских идиом на русский, чтобы не было мучительно больно</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/565874/">Собянин — амбасадор сервисов Яндекса

In [70]:
a_list = list(map(lambda x: x.find('a').get('href'), news_blocks))
a_list

['https://habr.com/ru/post/565904/',
 'https://habr.com/ru/post/565900/',
 'https://habr.com/ru/post/565892/',
 'https://habr.com/ru/post/564992/',
 'https://habr.com/ru/company/englishdom/blog/565786/',
 'https://habr.com/ru/post/565874/',
 'https://habr.com/ru/company/otus/blog/565846/',
 'https://habr.com/ru/post/565870/',
 'https://habr.com/ru/post/565742/',
 'https://habr.com/ru/post/565866/',
 'https://habr.com/ru/company/ruvds/blog/564300/',
 'https://habr.com/ru/company/vk/blog/565862/',
 'https://habr.com/ru/post/565856/',
 'https://habr.com/ru/company/otus/blog/565832/',
 'https://habr.com/ru/company/adapty/blog/565848/',
 'https://habr.com/ru/company/aliexpress_russia/blog/565800/',
 'https://habr.com/ru/company/macloud/blog/565340/',
 'https://habr.com/ru/company/jetinfosystems/blog/565844/',
 'https://habr.com/ru/company/redmadrobot/blog/565828/',
 'https://habr.com/ru/post/565834/']

In [71]:
# объединим все в одну функцию
def get_all_links(url, query, pages):
    all_refs = []
    params = {
        'search_query': query,
    }
    for i in range(1, pages):
        params['page'] = i
        res = requests.get(url, params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        news_blocks = soup.find_all('h2', class_='post__title')
        a_list = list(map(lambda x: x.find('a').get('href'), news_blocks))

    return a_list

all_links = get_all_links(url, KEYWORDS, 10)
all_links

['https://habr.com/ru/post/565570/',
 'https://habr.com/ru/post/565914/',
 'https://habr.com/ru/company/qrator/blog/565906/',
 'https://habr.com/ru/post/565904/',
 'https://habr.com/ru/post/565900/',
 'https://habr.com/ru/post/565892/',
 'https://habr.com/ru/post/564992/',
 'https://habr.com/ru/company/englishdom/blog/565786/',
 'https://habr.com/ru/post/565874/',
 'https://habr.com/ru/company/otus/blog/565846/',
 'https://habr.com/ru/post/565870/',
 'https://habr.com/ru/post/565742/',
 'https://habr.com/ru/post/565866/',
 'https://habr.com/ru/company/ruvds/blog/564300/',
 'https://habr.com/ru/company/vk/blog/565862/',
 'https://habr.com/ru/post/565856/',
 'https://habr.com/ru/company/otus/blog/565832/',
 'https://habr.com/ru/company/adapty/blog/565848/',
 'https://habr.com/ru/company/aliexpress_russia/blog/565800/',
 'https://habr.com/ru/company/macloud/blog/565340/']

In [12]:
import time
time.sleep(0.2)

In [65]:
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.2)
    if soup.find('span', class_='news-topic__attr news-topic__attr_date-time'):
        date = soup.find('span', class_='news-topic__attr news-topic__attr_date-time').text()
    else:
        date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()  
    print(date)
    if soup.find('a', class_='post_title_link'): 
        title = soup.find('a', class_='post__title_link').text
    else: 
        title = soup.find('span', class_='post__title-text').text    
    print(title)
    
    if soup.find('div', class_='post__text post__text-html  post__text_v2'):
        text = soup.find('div', class_='post__text post__text-html  post__text_v2').text
    elif soup.find('div', class_='post__text post__text-html post__text_v1'):
        text = soup.find('div', class_='post__text post__text-html post__text_v1').text
    else:
        text = soup.find('div', class_='post__text post__text_v2').text 
    print(text)
    print('')

2021-07-02
Как у меня увели домен. Продолжение
Начало истории читайте тут https://habr.com/ru/post/546174/У меня угнали домен, просто подделав цифровую подпись. Если регистратор предоставляет возможность переноса домена другому лицу по цифровой подписи, то защититься от этого никак нельзя.Регистратор REG.RU, спасибо ему хоть нам этом, прислал мне всю информацию которую ему предоставил злоумышленник. Из интересного:1) Скан поддельного паспорта 1 в 1 мой, вся информацию в нем моя. То есть кто-то продал информацию моего паспорта целиком и полностью. Если бы там была только 1ая страница и прописка, но там просто все! Отличия: моя подпись и подпись того кто выдавал паспорт другие. Было написано г. Горький Нижегородская область =) По этому вот скану я лишился домена.2) В дальнейшем оказалось, что в гос услугах отображалась другая цифровая подпись. Кем и для чего она была создана, увы узнать не удалось. Регистратор же предоставил данные той, которая не отображалась в гос услугах. Теперь даже 

2021-07-02
Почему иногда React/Redux в текущем состоянии give me creeps
React существует достаточно давно, чтобы мажорные изменения в этой библиотеке, не ощущались температурой подогрева кресел разработчиков в холодные зимние вечера (не благодарите за лайфхак). Но Facebook сделали ход конем и в свое время выпустили не мажорную, а минорную версию и тем самым сняли с себя ответственность за нестабильность уже существующих миллионов репозиториев, как вы уже поняли я буду рассказывать про версию 16.8.0, а так как мы почти никогда не используем React без Redux в продакшн репозиторияx, то и про него скажу. И сперва давайте поговорим про React. Почему была упомянута нестабильность после внесения “дополнений” 16.8.0, проблема в том что она произошла в головах разработчиков - легким движением руки Facebook сказал нам, знаете, ООП это конечно же хорошо, но функциональный подход лучше. И тут особо ярые и продвинутые ринулись кидать уже существующий подход Statefull Components и Stateless Componen

2021-07-02
Covid fake FAQ___draft_final_4 (окончательное доказательство)
Как говорил доктор Хаус: Все врут.Я решил написать данный текст после прочтения недавнего поста на хабре от "скептика" а так же после длительных дискуссий с разными людьми, которые уверены что все это фейк и даже больше. В данном тексте я не оцениваю политизированность темы, ошибочность принятых решений или необходимость каких-то действий. Цель данного текста исключительно в развеивании мифов, которыми люди бросаются либо из-за невежества, лени, злости или с целью ввести оппонента в заблуждение. Я так же стараюсь не касаться вопросов кому это нужно и какова цель, так как прекрасно понимаю, что если у вас параноя, то это не значит, что за вами не следят.Текст разделен на две части: Ковид в частности и вирусология в общем.Если у вас есть дополнения или исправления, пожалуйста - пишите их в комментариях с аргументацией и, если необходимо с приведением источников.Если у вас есть вопрос или утверждение которое вы хотит

2021-07-02
Королевские рефакторинги с Roslyn
Всем привет! Давно хотел написать эту статью, но материал всё никак не собирался: всё-таки рефакторинг - не на каждый день развлечение. Особенно если крупный. Речь пойдёт о том, как можно использовать Roslyn для лечения разной боли в шарповом коде. Если по центру вашего проекта возвышается огромная куча неудобного легаси и вам страшно на неё смотреть - добро пожаловать под кат. Возможно, мой материал позволит взглянуть на проблему с другого ракурса и понять, что не всё так печально. По идее сам подход может быть применим к любому языку, экосистема которого включает в себя тулинг для компилятора. Но это не точно. Однако, я расскажу всё, что знаю на двух примерах из реальной жизни.ПредысторияЧто вообще такое легаси? Дословно это переводится "наследие". В контексте - куча кода, которая написана давно, на старых технологиях, с использованием старых, глупых или попросту странноватых подходов. Все понимают что легаси-код - это плохо, но никто ниче

2021-07-02
5 способов перевода английских идиом на русский, чтобы не было мучительно больно
С идиомами в английском языке огромное количество конфузов. Их тысячи — и далеко не все из них можно хотя бы нормально объяснить на русском. Сегодня мы поговорим именно о переводе идиом. Расскажем о пяти тактиках, как их можно переводить, чтобы они нормально звучали.Откуда берутся идиомы: исторический экскурсИдиома — это особое словосочетание или фраза, устойчивое выражение, смысл которого не совпадает со смыслом слов, входящих в него.Буквальный перевод с идиомами не работает. Ну, или почти не работает. Даже если вы умеете пользоваться логикой, во многих случаях она вам не поможет.Есть отличное пожелание «Break a leg». Дословно переводится как «сломай ногу». Но на самом деле оно означает «Удачи». Сейчас это кажется бессмыслицей, но если посмотреть на происхождение идиомы, все становится более понятным. В британских театрах в конце XIX и начале XX века платили только тем актерам, которые играли н

...

2021-07-02
Архитектура контейнеров, часть 1. Почему важно понимать разницу между пространством пользователя и пространством ядра
Материал переведен. Ссылка на оригинальную статьюВам поручили спроектировать инфраструктуру на основе контейнеров? И вы, скорее всего, понимаете, какую пользу могут принести контейнеры разработчикам, архитекторам и командам эксплуатации. Вы уже что-то читали о них и с нетерпением ждете возможности более подробно изучить эту технологию. Однако перед погружением в обсуждение архитектуры и развертывание контейнеров в продакшн-окружении необходимо знать три важные вещи:Все приложения, включая контейнерные, используют ядро базовой ОС.Ядро предоставляет приложениям API через системные вызовы (system calls).Важны версии этого API, так как это "клей", который обеспечивает детерминированное взаимодействие между пространством пользователя (user space) и пространством ядра (kernel space).Иногда контейнеры рассматриваются как виртуальные машины, но важно отметить, что, в

2021-07-02
Сергей Маслов из Prisma Labs: разговор о мобильных приложениях, подписках, триалах и атрибуции
Мы запустили подкаст, в котором обсуждаем бизнес мобильных приложений: монетизацию, подписку, маркетинг, тенденции и актуальные изменения. Мы зовём топовых гостей, чей опыт полезен и нам самим, и нашим слушателям.В первом выпуске разговариваем с Сергеем Масловым из Prisma Labs.Послушать подкаст можно там, где вам удобно:YoutubeAppleYandexSound CloudО подкасте, госте и продукте, с которым он работаетВиталий Давыдов: Всем привет, я Виталик Давыдов.Никита Майданов: Привет-привет, я Никита Майданов.Виталий Давыдов: Мы запускаем новые подкаст о мобильных подписках SubHub. Мы будем рассказывать обо всем, что касается мобильных подписок, монетизации, и приглашать выдающихся гостей к нам, чтобы вы услышали их истории роста их мобильных приложений. Подкаст создается при поддержке Adapty. Мы делаем этот сервис для аналитики и роста мобильных приложений с подписками. С помощью Adapty разработ

2021-07-02
Экзорцист со светоотражающим звуком на солнечной батарее: что не так с переводами на AliExpress
Привет. Меня зовут Владимир, я работаю в «AliExpress Россия» специалистом по локализации.AliExpress известен своими странными переводами. Сколько шуток было на эту тему, даже от нас самих. И сторонний наблюдатель восклицает: «Ну сделайте уже нормально!»; продвинутый технарь ухмыляется: «Не могут нормальный Гугл Транслейт прикрутить»; а профессионалы из области машинного перевода наперебой советуют «кастомизированный движок» или MTPE. Все они и правы, и неправы одновременно.В этой статье я расскажу:откуда в AliExpress берутся тексты, на какие виды делятся и кто за них отвечает;почему переводят не с китайского, а виноват всё равно он;что такое Alibaba Translate и чем он лучше Google Translate или «Яндекс.Переводчика»;какие стратегии применяются для улучшения переводов и что ещё предстоит сделать.Но для того чтобы понять сегодняшние проблемы, придется зайти издалека.TL;DRЕсли вам нек

2021-07-02
Пар против бензина

О чём подумает обыватель услышав о паровом автомобиле? Наверное, на ум придут неспешные паровозы, громоздкие котлы, закипающие по полчаса, да клубы пара и дыма. Паровая машина ассоциируется с чем угодно, но только не с образом небольшого проворного автомобиля, по виду которого и не скажешь, что под капотом у него нет привычного ДВС.

Какое-то время автомобили с электродвигателями, паровыми машинами и ДВС успешно конкурировали друг с другом на рынке. В 1902 году в Америке из 909 новых зарегистрированных автомобилей 485 были паровыми. Но уже к началу 1910-х конструкторы смогли значительно улучшить ДВС введя такие новшества, как электростартер. Эпоха пара стремительно заканчивалась. Самые совершенные паровые автомобили 1920-х годов совершили свой последний рывок, но не смогли составить конкуренцию массовым и недорогим карбюраторным машинам. О них мы сегодня и поговорим.


Поздний Stenley Steamer с паровым двигателем и конденсатором

Чем хорош пар
Может показ

2021-07-02
ТОП-3 ИБ-событий недели по версии Jet CSIRT


Сегодня пятница, а значит, специалисты Jet CSIRT снова собрали для вас ключевые новости в области ИБ. В этот раз в ТОП-3 — новые атаки шифровальщика Babuk Locker, уязвимость в диспетчере очереди печати Windows и киберкампания, направленная на госструктуры Азиатских стран. Новости выбирал Игорь Фиц, аналитик центра мониторинга и реагирования на инциденты ИБ Jet CSIRT компании «Инфосистемы Джет».
Подробнее читайте под катом.

Конструктор вымогателя Babuk Locker используется в новых атаках
На прошлой неделе исследователь безопасности Кевин Бомонт обнаружил, что кто-то загрузил программу-вымогатель Babuk Locker на VirusTotal. После того, как конструктор просочился в сеть, некий злоумышленник начал применять его в атаках на обычных пользователей. Хакер использует адрес электронной почты Tutanota и требует выкуп в размере 210 долларов. Хотим отметить, что создатели Babuk Locker объявили о прекращении операций с вымогателем после громко

In [75]:
# запишем данные в датафрейм
habr_news = pd.DataFrame()
def get_all_links(url, query, pages):
    all_refs = []
    params = {
            'search_query': query,
        }
    for i in range(1, pages):
            params['page'] = i
            res = requests.get(url, params)
            soup = BeautifulSoup(res.text, 'html.parser')
            news_blocks = soup.find_all('h2', class_='post__title')
            a_list = list(map(lambda x: x.find('a').get('href'), news_blocks))

    return a_list

    all_links = get_all_links(url, KEYWORDS, 10)



for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    if soup.find('span', class_='news-topic__attr news-topic__attr_date-time'):
        date = soup.find('span', class_='news-topic__attr news-topic__attr_date-time').text()
    else:
        date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()  
    if soup.find('a', class_='post_title_link'): 
        title = soup.find('a', class_='post__title_link').text
    else:
        title = soup.find('span', class_='post__title-text').text    
    if soup.find('div', class_='post__text post__text-html  post__text_v2'):
        text = soup.find('div', class_='post__text post__text-html  post__text_v2').text
    elif soup.find('div', class_='post__text post__text-html post__text_v1'):
        text = soup.find('div', class_='post__text post__text-html post__text_v1').text
    else:
        text = soup.find('div', class_='post__text post__text_v2').text 
    row = {'date': date, 'title': title,'all_links': all_links, 'text': text}
    habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
habr_news.reset_index()

,index,date,title,all_links,text
0,0,2021-07-02,"Антивакцинаторы и теория игр, или математическ...","[https://habr.com/ru/post/565570/, https://hab...","\nНесмотря на злободневность, идея этой заметк..."
1,0,2021-07-02,Как у меня увели домен. Продолжение,"[https://habr.com/ru/post/565570/, https://hab...",Начало истории читайте тут https://habr.com/ru...
2,0,2021-07-02,Измерение интенсивности трафика при помощи u-м...,"[https://habr.com/ru/post/565570/, https://hab...",\nИзмерение интенсивности потоков в представле...
3,0,2021-07-02,Почему иногда React/Redux в текущем состоянии ...,"[https://habr.com/ru/post/565570/, https://hab...","React существует достаточно давно, чтобы мажор..."
4,0,2021-07-02,Хроника будущего: как будет эволюционировать ч...,"[https://habr.com/ru/post/565570/, https://hab...",Данный пост является саммари книги «‎All Tomor...
5,0,2021-07-02,Covid fake FAQ___draft_final_4 (окончательное ...,"[https://habr.com/ru/post/565570/, https://hab...",Как говорил доктор Хаус: Все врут.Я решил напи...
6,0,2021-07-02,Королевские рефакторинги с Roslyn,"[https://habr.com/ru/post/565570/, https://hab...","Всем привет! Давно хотел написать эту статью, ..."
7,0,2021-07-02,5 способов перевода английских идиом на русски...,"[https://habr.com/ru/post/565570/, https://hab...",С идиомами в английском языке огромное количес...
8,0,2021-07-02,Собянин — амбасадор сервисов Яндекса или как з...,"[https://habr.com/ru/post/565570/, https://hab...",На днях нам в предложку упал рассказ от одного...
9,0,2021-07-02,Проектирование программного обеспечения: что т...,"[https://habr.com/ru/post/565570/, https://hab...",Вы разрабатываете функцию для веб-сайта. Пусть...


In [68]:
def get_habr_news(links):
    habr_news = pd.DataFrame()
    for link in hubs:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        if soup.find('div', class_='b-article__publish_date'):
            date = pd.to_datetime(soup.find('div', class_='b-article__publish_date').find('time').get('datetime'), dayfirst=True).date()
        elif soup.find('time', class_='title__cake'):
            date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
        if soup.find('h2', class_='article_name'): 
            title = soup.find('h2', class_='article_name').text
        else: 
            title = soup.find('h1', class_='article_name').text    
        text = soup.find('div', class_='article_text_wrapper').text
        row = {'date': date, 'title': title, 'text': text}
        habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
    return habr_news
habr_news.reset_index()

,index,date,title,text
0,0,2021-07-02,Как у меня увели домен. Продолжение,Начало истории читайте тут https://habr.com/ru...
1,0,2021-07-02,Измерение интенсивности трафика при помощи u-м...,\nИзмерение интенсивности потоков в представле...
2,0,2021-07-02,Почему иногда React/Redux в текущем состоянии ...,"React существует достаточно давно, чтобы мажор..."
3,0,2021-07-02,Хроника будущего: как будет эволюционировать ч...,Данный пост является саммари книги «‎All Tomor...
4,0,2021-07-02,Covid fake FAQ___draft_final_4 (окончательное ...,Как говорил доктор Хаус: Все врут.Я решил напи...
5,0,2021-07-02,Королевские рефакторинги с Roslyn,"Всем привет! Давно хотел написать эту статью, ..."
6,0,2021-07-02,5 способов перевода английских идиом на русски...,С идиомами в английском языке огромное количес...
7,0,2021-07-02,Собянин — амбасадор сервисов Яндекса или как з...,На днях нам в предложку упал рассказ от одного...
8,0,2021-07-02,Проектирование программного обеспечения: что т...,Вы разрабатываете функцию для веб-сайта. Пусть...
9,0,2021-07-02,"Книжный рынок мёртв, а я ещё нет","Я читатель со стажем. Можно сказать, что я чит..."


# Задание 2. 

Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

